## IPL Win Predictor

### Data Parsing and Cleaning

In [98]:
# importing the libraries
import numpy as np
import pandas as pd

In [99]:
# importing the dataset
match = pd.read_csv('matches.csv')
delivery = pd.read_csv('deliveries.csv')

In [100]:
match.head()

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [101]:
match.shape

(756, 18)

In [102]:
delivery.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [103]:
#calculating total runs inning wise in each match by using group by
total_score_df = delivery.groupby(['match_id', 'inning']).sum()['total_runs'].reset_index()

C:\Users\Saket\AppData\Local\Temp\ipykernel_3852\3980492545.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_score_df = delivery.groupby(['match_id', 'inning']).sum()['total_runs'].reset_index()


In [104]:
# removing scores of the seconfd innings as not required
total_score_df = total_score_df[total_score_df['inning'] == 1]

In [105]:
# merging total_score_df in matches dataset
match_df = match.merge(total_score_df[['match_id', 'total_runs']],left_on="id", right_on="match_id")

#### Removing teams which are not playing and chainging names of team whose names are changed

In [106]:
match_df['team1'].unique()

array(['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab',
       'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants',
       'Delhi Capitals'], dtype=object)

In [107]:
match_df.shape

(756, 20)

In [108]:
# making list of currently playing teams
teams = [
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders'
    'Kings XI Punjab',
    'Chennai Super Kings',
    'Rajasthan Royals',
    'Delhi Capitals'
]

In [109]:
# replacing team names for delhi and heydrabad
match_df['team1'] = match_df['team1'].str.replace('Delhi Daredevils', 'Delhi Capitals')
match_df['team2'] = match_df['team2'].str.replace('Delhi Daredevils', 'Delhi Capitals')

match_df['team1'] = match_df['team1'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')
match_df['team2'] = match_df['team2'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')



In [110]:
# removing all other teams which are not playing
match_df = match_df[match_df['team1'].isin(teams)]
match_df = match_df[match_df['team2'].isin(teams)]

In [111]:
match_df.shape

(342, 20)

In [112]:
# removing matches that were affected by rains
match_df = match_df[match_df['dl_applied'] == 0]

In [113]:
# extracting cols which are needed from match_df
match_df = match_df[['match_id', 'city', 'winner', 'total_runs' ]]

In [114]:
# assigning match-dfwith delivery
delivery_df = match_df.merge(delivery, on='match_id')

In [115]:
#for chasing innings data
delivery_df = delivery_df[delivery_df['inning'] == 2]

In [116]:
#Cummilative Sum function
delivery_df['current_score'] = delivery_df.groupby('match_id')['total_runs_y'].cumsum()

In [117]:
#Runs Left
delivery_df['runs_left'] = delivery_df['total_runs_x'] - delivery_df['current_score']

In [118]:
#Balls Left
delivery_df['balls_left'] = 126 - (delivery_df['over']*6 + delivery_df['ball'])

In [119]:
#Wickets Left
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].fillna("0")
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].apply(lambda x:x if x == "0" else "1")
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].astype('int')
wickets = delivery_df.groupby('match_id')['player_dismissed'].cumsum().values
delivery_df['wickets'] = 10 - wickets
delivery_df.head()

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_left,balls_left,wickets
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,1,0,1,0,NaN,NaN,1,206,119,10
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,0,NaN,NaN,1,206,118,10
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,0,0,0,0,NaN,NaN,1,206,117,10
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,2,0,2,0,NaN,NaN,3,204,116,10
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,...,4,0,4,0,NaN,NaN,7,200,115,10


In [120]:
#Current Run Rate
delivery_df['crr'] = (delivery_df['current_score']*6)/(120 - delivery_df['balls_left'])

In [121]:
#Required Run Rate
delivery_df['rrr'] = (delivery_df['runs_left']*6)/delivery_df['balls_left']

In [122]:
#Result
def result(row):
    return 1 if row['batting_team'] == row['winner'] else 0

In [123]:
delivery_df['result'] = delivery_df.apply(result,axis=1)

In [124]:
final_df = delivery_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets','total_runs_x','crr','rrr','result']]

In [125]:
final_df.sample(final_df.shape[0])

,batting_team,bowling_team,city,runs_left,balls_left,wickets,total_runs_x,crr,rrr,result
23966,Royal Challengers Bangalore,Deccan Chargers,Nagpur,55,38,7,151,7.024390,8.684211,0
76097,Rajasthan Royals,Mumbai Indians,Mumbai,191,117,10,191,0.000000,9.794872,1
76445,Sunrisers Hyderabad,Delhi Capitals,Hyderabad,51,15,3,164,6.457143,20.400000,0
51943,Mumbai Indians,Sunrisers Hyderabad,Hyderabad,15,15,7,157,8.114286,6.000000,1
3053,Royal Challengers Bangalore,Mumbai Indians,Mumbai,97,67,9,165,7.698113,8.686567,1
...,...,...,...,...,...,...,...,...,...,...
713,Mumbai Indians,Sunrisers Hyderabad,Mumbai,36,29,6,158,8.043956,7.448276,1
63790,Royal Challengers Bangalore,Sunrisers Hyderabad,Hyderabad,57,28,6,194,8.934783,12.214286,0
70110,Royal Challengers Bangalore,Sunrisers Hyderabad,Hyderabad,31,20,5,155,7.440000,9.300000,0
37572,Chennai Super Kings,Delhi Daredevils,Chennai,70,88,10,114,8.250000,4.772727,1


In [126]:
X = final_df.iloc[:,:-1]
y = final_df.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=1)

In [127]:
X_train

,batting_team,bowling_team,city,runs_left,balls_left,wickets,total_runs_x,crr,rrr
12375,Rajasthan Royals,Delhi Daredevils,Centurion,30,23,5,143,6.989691,7.826087
59979,Mumbai Indians,Chennai Super Kings,Chennai,127,98,10,158,8.454545,7.775510
53338,Sunrisers Hyderabad,Royal Challengers Bangalore,Hyderabad,126,99,10,160,9.714286,7.636364
46323,Rajasthan Royals,Sunrisers Hyderabad,Hyderabad,84,47,7,136,4.273973,10.723404
54744,Delhi Daredevils,Chennai Super Kings,Chennai,135,106,10,150,6.428571,7.641509
...,...,...,...,...,...,...,...,...,...
16221,Chennai Super Kings,Mumbai Indians,Port Elizabeth,41,29,7,147,6.989011,8.482759
67408,Royal Challengers Bangalore,Delhi Daredevils,Bengaluru,67,40,7,186,8.925000,10.050000
10877,Royal Challengers Bangalore,Chennai Super Kings,Port Elizabeth,123,61,5,179,5.694915,12.098361
25339,Royal Challengers Bangalore,Mumbai Indians,Mumbai,169,111,10,184,10.000000,9.135135


In [128]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
trf= ColumnTransformer([
   ('trf',OneHotEncode(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')


NameError: name 'OneHotEncode' is not defined